In [20]:
import csv
import json
import openai

openai.api_key = ""

In [21]:
PROMPT = """
You are a coach for high school debate. You want to classify judge paradigms based on whether the judge is "progressive" or "traditional". Progressive judges will entertain progressive arugments, such as kritiks (K's) and Theory shells and things like using music in the debate round.
If they do not have any restrictions on progressive arguments and are completely accepting, they should be rated a 5.
If they will vote for progressive arguments, but have a few stipulations, they should be rated a 4.
If they know what progressive arguments are and state they may vote on them, but it isn't their preference, they should be rated a 3.
If progressive argumentation isn't mentioned in their paradigm, they should be given a 2 if they seem to be technical otherwise (for example, they talk about flowing, being a former coach or debater).
If they have no technical experience, they should be given a 1.

Most judges will be 1s and 2s.

Rank the given paradigms from 1 to 5, with 1 being a very traditional judge (not receptive to progressive arguments) and 10 being a very progressive judge. Only give a number and nothing else.
"""

In [22]:
data = []

with open('paradigms.csv', 'r') as f:
    table = csv.reader(f)
    for row in table:
        data.append({"messages": [
            {
                "role": "system",
                "content": PROMPT
            },
            {
                "role": "user",
                "content": f"Classify the following paradigm:\n{row[0]}"
            },
            {
                "role": "assistant",
                "content": f"{row[2]}"
            }
        ]})

with open("parsed.jsonl", "w") as f:
    for entry in data:
        json.dump(entry, f)
        f.write('\n')

In [23]:
response = openai.files.create(
  file=open("parsed.jsonl", "rb"),
  purpose='fine-tune'
)

In [24]:
FILE_ID = response.id

In [25]:
response = openai.fine_tuning.jobs.create(training_file=FILE_ID, model="gpt-3.5-turbo")
print(response.id)

ftjob-kaYXXE4oxlmxnXzxWIUMTGsM


In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [27]:
MODEL = "ft:gpt-3.5-turbo-0613:debate-land::8OClXt5D"
chat = ChatOpenAI(model=MODEL, api_key=openai.api_key)

In [28]:
response = chat([
    SystemMessage(content=PROMPT),
    HumanMessage(content=f"Classify the following:\n\nI am a flow judge")
])

result = None

try:
    result = int(response.content)
except Exception:
    pass
print(result)

2
